## Set Up

this is using python 3.13 

In [33]:
%pip install -Uq "unstructured[all-docs]" pillow lxml pillow
%pip install -Uq chromadb tiktoken
%pip install -Uq langchain langchain-community langchain-openai langchain-groq
%pip install -Uq python_dotenv

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Setting the enviroment variables

In [ ]:
import os

# keys for the services we will use

# os.environ["OPENAI_API_KEY"] = "sk-..."
os.environ["GROQ_API_KEY"] = "sh...."
os.environ["LANGCHAIN_API_KEY"] = "sh..."
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "sh..."

## Extract the data

Extract the elements of the PDF that we will be able to use in the retrieval process. These elements can be: Text, Images, Tables, etc.

### Partition PDF tables, text, and images

In [35]:
from unstructured.partition.pdf import partition_pdf

output_path = "./docs/"
file_path = output_path + 'Respuestas_basicas.pdf' ## output_path + 'attention.pdf'  

# Reference: https://docs.unstructured.io/open-source/core-functionality/chunking
chunks = partition_pdf(
    filename=file_path,
    infer_table_structure=True,            # extract tables
    strategy="hi_res",                     # mandatory to infer tables

    extract_image_block_types=["Image"],   # Add 'Table' to list to extract image of tables
    # image_output_dir_path=output_path,   # if None, images and tables will saved in base64

    extract_image_block_to_payload=True,   # if true, will extract base64 for API usage

    chunking_strategy="by_title",          # or 'basic'
    max_characters=10000,                  # defaults to 500
    combine_text_under_n_chars=2000,       # defaults to 0
    new_after_n_chars=6000,

    # extract_images_in_pdf=True,          # deprecated
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

##  We get 2 types of elements from the partition_pdf function

In [36]:
# in this case beacuse the pdf does not have images we only get 1 type of element wich is text

set([str(type(el)) for el in chunks])

{"<class 'unstructured.documents.elements.CompositeElement'>"}

In [45]:
# Each CompositeElement containes a bunch of related elements.
# This makes it easy to use these elements together in a RAG pipeline.

chunks[0].metadata.orig_elements

In [ ]:
# This is what an extracted image looks like.
# It contains the base64 representation only because we set the param extract_image_block_to_payload=True


# In this case we are not going to use this beacuse the pdf does not have images

elements = chunks[3].metadata.orig_elements
chunk_images = [el for el in elements if 'Image' in str(type(el))]
chunk_images[0].to_dict()

### Separate extracted elements into tables, text, and images

In [ ]:
# separate tables from texts
tables = []
texts = []

for chunk in chunks:
    if "Table" in str(type(chunk)):
        tables.append(chunk)

    if "CompositeElement" in str(type((chunk))):
        texts.append(chunk)


In [ ]:
# Get the images from the CompositeElement objects
def get_images_base64(chunks):
    images_b64 = []
    for chunk in chunks:
        if "CompositeElement" in str(type(chunk)):
            chunk_els = chunk.metadata.orig_elements
            for el in chunk_els:
                if "Image" in str(type(el)):
                    images_b64.append(el.metadata.image_base64)
    return images_b64

images = get_images_base64(chunks)

## Check what the image looks like

In [ ]:
import base64
from IPython.display import Image, display

def display_base64_image(base64_code):
    # Decode the base64 string to binary
    image_data = base64.b64decode(base64_code)
    # Display the image
    display(Image(data=image_data))

display_base64_image(images[0])

## Summarize the data

Create a summary of each element extracted from the PDF. This summary will be vectorized and used in the retrieval process.

### Text and Table summaries

We don't need a multimodal model to generate the summaries of the tables and the text. I will use open source models available on Groq.

In [14]:
%pip install -Uq langchain-groq


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [47]:
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [48]:
# Prompt
prompt_text = """
You are an assistant tasked with summarizing tables and text.
Give a concise summary of the table or text.

Respond only with the summary, no additionnal comment.
Do not start your message by saying "Here is a summary" or anything like that.
Just give the summary as it is.

Table or text chunk: {element}

"""
prompt = ChatPromptTemplate.from_template(prompt_text)

# Summary chain
model = ChatGroq(temperature=0.5, model="llama-3.1-8b-instant")
summarize_chain = {"element": lambda x: x} | prompt | model | StrOutputParser()

In [ ]:
# Summarize text
text_summaries = summarize_chain.batch(texts, {"max_concurrency": 3})

# Summarize tables
tables_html = [table.metadata.text_as_html for table in tables]
table_summaries = summarize_chain.batch(tables_html, {"max_concurrency": 3})


[]

In [53]:
text_summaries

['El Tecnológico ofrece varios trámites relacionados con certificados, constancias, credenciales, cambio de carrera, titulación, exámenes, inscripciones a licenciatura y posgrado, planes de estudio e información sobre estudios. Los trámites incluyen certificados de terminación, constancias con calificación, credenciales extemporeanas, cambio de carrera, donación de libro de titulación, exámenes especiales y examenes de admisión. Los precios varían según el trámite, desde $40.00 hasta $5,800.00.',
 'The table outlines key information for students at ITLA Laguna, including enrollment periods, academic programs, course schedules, and campus services. It covers topics such as: enrollment periods for undergraduate and graduate programs, course and convocation schedules, language courses, extracurricular activities, credential replacement and issuance, and class schedules.']

## Load data and summaries to vectorstore

In [54]:
%pip install langchain-huggingface

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [21]:
%pip install custom_st

ERROR: Could not find a version that satisfies the requirement custom_st (from versions: none)
ERROR: No matching distribution found for custom_st

[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### Create the vectorstore

In [55]:
from langchain.vectorstores import Chroma
from langchain.storage import InMemoryStore
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

# The vectorstore to use to index the child chunks
vectorstore = Chroma(
    collection_name="multi_modal_rag",
    embedding_function=HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
)

# The storage layer for the parent documents
store = InMemoryStore()
id_key = "doc_id"

# The retriever (empty to start)
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    docstore=store,
    id_key=id_key,
)

### Load the summaries and link the to the original data

In [ ]:
%pip install langchain-community

In [57]:
import uuid
from langchain_core.documents import Document
# Add texts
doc_ids = [str(uuid.uuid4()) for _ in texts]
summary_texts = [
    Document(page_content=summary, metadata={id_key: doc_ids[i]}) for i, summary in enumerate(text_summaries)
]
retriever.vectorstore.add_documents(summary_texts)
retriever.docstore.mset(list(zip(doc_ids, texts)))

### Check retrieval

In [58]:
# Retrieve
docs = retriever.invoke(
    "Inscripcion"
)

Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


In [59]:
for doc in docs:
    print(str(doc) + "\n\n" + "-" * 80)

Apartados y Respuestas sobre Trámites

Certificados

● Certificado de terminación y duplicados: $1,000.00

● Certificado parcial: $750.00

● Proceso: Se debe pagar en caja dentro de las instalaciones del Tecnológico y el recibo se lleva a servicios escolares los lunes, miércoles o viernes de 10 AM a 3 PM.

Constancias

● Constancia con calificación: $70.00

● Constancia por semestre: $70.00

● Constancia de estudios: $50.00

● Cárdex: $70.00

● Constancia de título: $70.00

● Recuperación de contraseña: $70.00

Credenciales

● Trámites disponibles:

○ Extemporáneo: $70.00

○ Duplicado de credencial: $100.00

○ Credencial de biblioteca: $40.00

○ Resello extemporáneo: $70.00

Cambio de Carrera

● Revisión de equivalencia: $750.00

● Convalidación: $500.00

● Solicitud de cambio de carrera: $800.00

● Más información: Puede contactar a su coordinador a través de estos correos.

Titulación

● Donación de libro de titulación: $1,200.00

● Protocolo de titulación: $3,500.00

● Certificación

## RAG pipeline

In [26]:
%pip install huggingface_hub

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [60]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.messages import HumanMessage
from base64 import b64decode
from langchain_community.llms import HuggingFaceHub
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

def parse_docs(docs):
    """Split base64-encoded images and texts"""
    b64 = []
    text = []
    for doc in docs:
        try:
            b64decode(doc)
            b64.append(doc)
        except Exception:
            text.append(doc)
    return {"images": b64, "texts": text}

def build_prompt(kwargs):
    docs_by_type = kwargs["context"]
    user_question = kwargs["question"]

    context_text = "".join(text_element.text for text_element in docs_by_type["texts"])

    prompt_template = f"""
    Answer the question based only on the following context, which can include text, tables, and images.
    Context: {context_text}
    Question: {user_question}
    """

    prompt_content = [{"type": "text", "text": prompt_template}]

    if len(docs_by_type["images"]) > 0:
        for image in docs_by_type["images"]:
            prompt_content.append(
                {
                    "type": "image_url",
                    "image_url": {"url": f"data:image/jpeg;base64,{image}"},
                }
            )

    return ChatPromptTemplate.from_messages(
        [
            HumanMessage(content=prompt_content),
        ]
    )

# Configurar el modelo de Hugging Face
llm = HuggingFaceHub(
    repo_id="mistralai/Mistral-7B-Instruct-v0.2",  # Puedes cambiar el modelo aquí
    model_kwargs={"temperature": 0.5, "max_length": 512}
)

chain = (
    {
        "context": retriever | RunnableLambda(parse_docs),
        "question": RunnablePassthrough(),
    }
    | RunnableLambda(build_prompt)
    | llm
    | StrOutputParser()
)

chain_with_sources = {
    "context": retriever | RunnableLambda(parse_docs),
    "question": RunnablePassthrough(),
} | RunnablePassthrough().assign(
    response=(
        RunnableLambda(build_prompt)
        | llm
        | StrOutputParser()
    )
)


In [61]:
response = chain.invoke(
    "certificados?"
)

print(response)

Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2
/Users/carlosrobertorochatrejo/Developer/projects/.venv/lib/python3.13/site-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


Human: [{'type': 'text', 'text': '\n    Answer the question based only on the following context, which can include text, tables, and images.\n    Context: Apartados y Respuestas sobre Trámites\n\nCertificados\n\n● Certificado de terminación y duplicados: $1,000.00\n\n● Certificado parcial: $750.00\n\n● Proceso: Se debe pagar en caja dentro de las instalaciones del Tecnológico y el recibo se lleva a servicios escolares los lunes, miércoles o viernes de 10 AM a 3 PM.\n\nConstancias\n\n● Constancia con calificación: $70.00\n\n● Constancia por semestre: $70.00\n\n● Constancia de estudios: $50.00\n\n● Cárdex: $70.00\n\n● Constancia de título: $70.00\n\n● Recuperación de contraseña: $70.00\n\nCredenciales\n\n● Trámites disponibles:\n\n○ Extemporáneo: $70.00\n\n○ Duplicado de credencial: $100.00\n\n○ Credencial de biblioteca: $40.00\n\n○ Resello extemporáneo: $70.00\n\nCambio de Carrera\n\n● Revisión de equivalencia: $750.00\n\n● Convalidación: $500.00\n\n● Solicitud de cambio de carrera: $80

In [62]:
response = chain_with_sources.invoke(
    "Cuales son los certificados?"
)

print("Response:", response['response'])

print("\n\nContext:")
for text in response['context']['texts']:
    print(text.text)
    print("Page number: ", text.metadata.page_number)
    print("\n" + "-"*50 + "\n")
for image in response['context']['images']:
    display_base64_image(image)

Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2
/Users/carlosrobertorochatrejo/Developer/projects/.venv/lib/python3.13/site-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


Response: Human: [{'type': 'text', 'text': '\n    Answer the question based only on the following context, which can include text, tables, and images.\n    Context: Apartados y Respuestas sobre Trámites\n\nCertificados\n\n● Certificado de terminación y duplicados: $1,000.00\n\n● Certificado parcial: $750.00\n\n● Proceso: Se debe pagar en caja dentro de las instalaciones del Tecnológico y el recibo se lleva a servicios escolares los lunes, miércoles o viernes de 10 AM a 3 PM.\n\nConstancias\n\n● Constancia con calificación: $70.00\n\n● Constancia por semestre: $70.00\n\n● Constancia de estudios: $50.00\n\n● Cárdex: $70.00\n\n● Constancia de título: $70.00\n\n● Recuperación de contraseña: $70.00\n\nCredenciales\n\n● Trámites disponibles:\n\n○ Extemporáneo: $70.00\n\n○ Duplicado de credencial: $100.00\n\n○ Credencial de biblioteca: $40.00\n\n○ Resello extemporáneo: $70.00\n\nCambio de Carrera\n\n● Revisión de equivalencia: $750.00\n\n● Convalidación: $500.00\n\n● Solicitud de cambio de ca